In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math

pd.set_option('display.max_rows',50)

In [2]:
### séparation de la colonne date_receiveid en deux colonnes : only_hours_received only_date_received
def split_date_received(dataframe):
    dataframe[['only_date_received', 'only_hours_received']] = dataframe['date_received'].str.split(expand=True)
    return dataframe

In [10]:
def jours_unique_func(dataframe):
    jours_unique= []
    jours_unique = dataframe['only_date_received'].unique()
    return jours_unique

In [4]:
def server_by_queuename_by_day (dataframe, jours_unique): #dataframe splité
    dic_jour = {} 
    for i in jours_unique:
        dic_service ={}
        list_service=[]

        # récupération de toute les lignes avec only_date_received = i
        df_jours_uniq = dataframe[dataframe.only_date_received==i]   # est un dataframe
        for j in df_jours_uniq.index :
            queue_name = df_jours_uniq["queue_name"][j] # le type de service
            if queue_name not in dic_service:
                agent_number = df_jours_uniq[df_jours_uniq.queue_name == queue_name]["agent_number"] # ici, c'est une série
                agent_deploye = []
                for ag in agent_number:
                    if ag not in agent_deploye and math.isnan(ag) == False: # afin d'exclure les valeurs manquante de agent_number
                        agent_deploye.append(ag)

                count_distinct_agent = len(agent_deploye)  
                #agent_deploye.clear()
                agent_deploye.append(count_distinct_agent)
                if count_distinct_agent!=0:
                    dic_service[queue_name] = count_distinct_agent

        list_service.append(dic_service)
        dic_jour[i]= list_service     

    return dic_jour

In [5]:
def browse_dictionnaire(dictionnaire):
    number_server_TS = pd.DataFrame()
    for key, value in dictionnaire.items():
        valeur = value[0]
        taille_df = len(valeur)
        list_date=[]
        for i in range(taille_df):        
            list_date.append(key)

        df_date = pd.DataFrame(list_date,columns=['date'])
        df_server = pd.DataFrame(list(valeur.items()),columns=['type_service', 'Nombre_serveur'])


        df_info_server_day = pd.concat([df_date, df_server], axis=1, join='inner')
        number_server_TS = pd.concat([number_server_TS, df_info_server_day], ignore_index=True, sort=False)
    
    return number_server_TS

In [6]:
# Ajout de colonne serveur au dataframe
def add_col_serveur(dataframe, df_browse_dictionnaire): # df_split
    dataframe["serveur"]= ""
    for k in range(len(dataframe)):
        queue_name = dataframe.loc[k,"queue_name"]
        only_date_received = dataframe.loc[k,"only_date_received"]
        agent_number= dataframe.loc[k,"agent_number"]
        # récupérer le nombre de serveur pour queue_name le jour only_date_received
        serveur = df_browse_dictionnaire[(df_browse_dictionnaire.date==only_date_received) & (df_browse_dictionnaire.type_service==queue_name)]
        serveur=serveur["Nombre_serveur"]
        for i in serveur:
            s=i
            dataframe["serveur"][k]=s 
    return dataframe

In [11]:
def function_finale(df_input):
    df_output_1 = split_date_received(df_input)
    jours_unique = jours_unique_func(df_input)
    dictionnaire = server_by_queuename_by_day(df_output_1,jours_unique) # il sort un dictionnaire
    df_browse_dictionnaire = browse_dictionnaire(dictionnaire)
    df_output_finale=add_col_serveur(df_output_1, df_browse_dictionnaire)
    return df_output_finale

In [12]:
# sauveagrde dans un fichier csv
df_01 = pd.read_csv('dataset/calls-2014-01.csv')
df_output_finale = function_finale(df_01)


C:\Users\HP\AppData\Local\Temp\ipykernel_12976\3047877329.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["serveur"][k]=s


In [13]:
# importation du jeu de données calls-2014-01.csv
df_output_finale.head()

,date_received,queue_name,agent_number,answered,consult,transfer,hangup,only_date_received,only_hours_received,serveur
0,2014-01-02 08:03:21,30175,6935.0,2014-01-02 08:03:22,NaN,NaN,2014-01-02 08:05:54,2014-01-02,08:03:21,78
1,2014-01-02 08:04:37,30560,6940.0,2014-01-02 08:04:38,NaN,NaN,2014-01-02 08:09:49,2014-01-02,08:04:37,64
2,2014-01-02 08:06:26,30172,6968.0,2014-01-02 08:06:27,NaN,NaN,2014-01-02 08:15:17,2014-01-02,08:06:26,62
3,2014-01-02 08:08:07,30175,9424.0,2014-01-02 08:08:08,NaN,NaN,2014-01-02 08:10:37,2014-01-02,08:08:07,78
4,2014-01-02 08:08:26,30172,8076.0,2014-01-02 08:08:27,NaN,NaN,2014-01-02 08:14:59,2014-01-02,08:08:26,62


In [14]:
# Sauvegarde dans un fichier csv
df_output_finale.to_csv('df_01_output.csv', index=False)